# Inference NAO

In this colab, we will describe the GenFocal debiased and downscaled forecasts for the North Atlantic Ocean (NAO) hurricane basin for the months of August, September, and October for 2010-2019.  These datasets
contain the following variables
 - 10mW, wind speed at 10 meters (m/s)
 - 2mT, temperature at 2 meters (K)
 - MSL, mean sea-level pressure (Pa)
 - Q1000, near-surface specific humidity (kg/kg)
 - Z200, geopotential at 200 hPa (m)
 - Z500, geopotential at 500 hPa (m)

In [ ]:
# @title PIP Installs
!pip install -q zarr xarray[complete] fsspec aiohttp requests gcsfs cartopy \
  cfgrib eccodes cf_xarray pint_xarray


In [ ]:
# @title Imports
import h5py
import gcsfs
import matplotlib.pyplot as plt
from google.colab import auth
from google.cloud import storage
from datetime import datetime
import pandas as pd
import numpy as np
from cartopy import config
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import cf_xarray.units
import pint_xarray


In [ ]:
# @title Plotting Functions

def plot_scalars(temp_data1, temp_data2, lat_min, lat_max, lon_min, lon_max):
    """
    Plots two scalar arrays on the same plot with a shared colorbar.

    Args:
        temp_data1: xarray DataArray of the first temperature data.
        temp_data2: xarray DataArray of the second temperature data.
        lat_min: Minimum latitude for the plot.
        lat_max: Maximum latitude for the plot.
        lon_min: Minimum longitude for the plot.
        lon_max: Maximum longitude for the plot.
    """

    fig, axs = plt.subplots(nrows=1, ncols=2,
                            subplot_kw={'projection': ccrs.PlateCarree()},
                            figsize=(12, 6))

    # Combine data for shared colorbar limits
    vmin = min(temp_data1.min(), temp_data2.min())
    vmax = max(temp_data1.max(), temp_data2.max())

    # Plot the first temperature data
    im1 = temp_data1.plot(
        ax=axs[0], transform=ccrs.PlateCarree(), add_colorbar=False,
           x='longitude', y='latitude',
           vmin=vmin, vmax=vmax,
           cmap='viridis'
    )

    # Overlay the second temperature data
    im2 = temp_data2.plot(
        ax=axs[1], transform=ccrs.PlateCarree(), add_colorbar=False,
           x='longitude', y='latitude',
           vmin=vmin, vmax=vmax,
           cmap='viridis'
    )


    # Add coastlines and gridlines
    for ax in axs:
      ax.coastlines()
      ax.add_feature(cfeature.BORDERS)
      ax.add_feature(cfeature.RIVERS)
      # Set plot extent
      ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
      cbar = plt.colorbar(im1, ax=ax, shrink=0.7)  # Use im1 for the colorbar
      cbar.set_label('Temperature (K)')

    plt.show()

def plot_scalar(temp_data1, lat_min, lat_max, lon_min, lon_max, title_string=""):
    """
    Plots two scalar arrays on the same plot with a shared colorbar.

    Args:
        temp_data1: xarray DataArray of the first temperature data.
        temp_data2: xarray DataArray of the second temperature data.
        lat_min: Minimum latitude for the plot.
        lat_max: Maximum latitude for the plot.
        lon_min: Minimum longitude for the plot.
        lon_max: Maximum longitude for the plot.
    """

    fig, axs = plt.subplots(nrows=1, ncols=1,
                            subplot_kw={'projection': ccrs.PlateCarree()},
                            figsize=(12, 6))

    vmin = temp_data1.min()
    vmax = temp_data1.max()

    # Plot the temperature data
    im1 = temp_data1.plot(
        ax=axs, transform=ccrs.PlateCarree(), add_colorbar=False,
           x='longitude', y='latitude',
           vmin=vmin, vmax=vmax,
           cmap='viridis'
    )

    # Add coastlines and gridlines
    axs.coastlines()
    axs.add_feature(cfeature.BORDERS)
    axs.add_feature(cfeature.STATES)
    # Set plot extent
    axs.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
    cbar = plt.colorbar(im1, ax=axs, shrink=0.7)  # Use im1 for the colorbar
    cbar.set_label('Temperature (K)')
    plt.title(title_string)

    plt.show()


We now need to authenticate with Google Cloud so we can access the GenFocal bucket

In [ ]:
auth.authenticate_user()

# Insert Text

In [ ]:
inference_nao = xr.open_zarr(
            "gs://genfocal/data/inference/nao/debiased_100members_aso10s_8samples_xm155030721_member_chunks.zarr",
            consolidated=True)
inference_nao

In [ ]:
# @title Plot an individual ensemble member
surface_variable_name = "10mW" # @param dropdown ["MSL", "10mW"]
ensemble_member = 0 # @param {type:"integer"}
date = "2015-09-01" # @param {type:"date"}
hour = 0 # @param {type:"integer")
if (hour <0) :
  hour = 0
if (hour >=22) :
  hour = 22

latgrid = inference_nao.latitude
longrid = inference_nao.longitude

title_string = f"{surface_variable_name} for {date} member {ensemble_member}"
time_slice=slice(f"{date} T{hour:02d}", f"{date} T{hour:02d}")
scalar_array_daily_member = inference_nao[surface_variable_name].sel(time=time_slice).sel(member=ensemble_member).squeeze().compute()
plot_scalar(scalar_array_daily_member, latgrid.min(), latgrid.max(),
             longrid.min(), longrid.max(), title_string)

In [ ]:
# @title Ensemble mean of daily minimum MSL pressure
surface_variable_name = "MSL"
date = "2015-09-01" # @param {type:"date"}
time_slice=slice(f"{date} T00", f"{date} T23")

scalar_array_daily = inference_nao[surface_variable_name].sel(time=time_slice).squeeze()
latgrid = inference_nao.latitude
longrid = inference_nao.longitude
title_string = f"Ensemble mean of daily minimum MSL pressure for {date}"
scalar_array_dailymn_mean = scalar_array_daily.min(dim='time').mean(dim='member').compute()
plot_scalar(scalar_array_dailymn_mean, latgrid.min(), latgrid.max(),
             longrid.min(), longrid.max(), title_string)


In [ ]:
# @title Ensemble standard deviation of daily minimum MSL pressure
scalar_array_daily_std = scalar_array_daily.min(dim='time').std(dim='member').compute()
title_string = f"Ensemble standard deviation of daily minimum MSL pressure for {date}"
plot_scalar(scalar_array_daily_std, latgrid.min(), latgrid.max(),
             longrid.min(), longrid.max(), title_string)
